## COP and COG estimation by derivative method
Trying to estimate COP and COG were used linear function wich didn't give the results expected, therefore we tried to make different approaches and one of them is the next function:
COP= COG-(4/g*3*d²derivadasegundaTeta)

## Python libraries

First, let's import the necessary Python libraries and configure the environment:

In [ ]:
import numpy as np
#from scipy.signal import butter, filtfilt
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
#import seaborn as sns
#sns.set_context("notebook", font_scale=1.3,
#                rc={'lines.linewidth': 1.5, 'lines.markersize': 14, 'axes.titlesize': 'x-large'})
#matplotlib.rc('legend', numpoints=1, fontsize=14)
#import glob
import sys, os
sys.path.insert(1, r'./Functions')
# IPython widgets:
from IPython.display import display
import ipywidgets
from ipywidgets import FloatProgress, interactive

Versions of the Python libraries used:

In [ ]:
#!pip install version_information
%load_ext version_information
%version_information numpy, scipy, pandas, matplotlib, seaborn

## Meta data

The file `PDSinfo.txt` contains meta data about the subjects and the experimental trials. The file has a header plus 588 rows per 29 columns (there are 12 rows for each of the 49 subjects).   
Let's use the power of the [pandas](http://pandas.pydata.org/) library to load and explore the meta data:

In [ ]:
# GitHub URL:
#path2 = 'C:\Users\ferna\Downloads\IC- dados\Dados'
# local directory:
path2 = r'C:\\Users\\ferna\\Downloads\\IC- dados\\Dados'
fname = os.path.join(path2, 'PDSinfo.txt')
PDSinfo = pd.read_csv(fname, sep='\t', header=0, index_col=None, engine='c', encoding='utf-8')
print(fname)
print("Information of %s subjects loaded (%s rows, %s columns)."
      %(len(pd.unique(PDSinfo.Subject)), PDSinfo.shape[0], PDSinfo.shape[1]))

In [ ]:
a=PDSinfo[PDSinfo['Subject']==44].index.tolist()
e=PDSinfo[PDSinfo['Subject']==44].index.tolist()[2]
print(a)
print(e)

Here are the first 12 rows and first 10 columns of meta data:

## COP and COG displacements

## For Every Subject

The Center of Pressure mesured will be called COPXrealmatrix and COPZrealmatrix and the one calculated by COPXmatrix and COPZmatrix by the equation with Yp being the center of pressure. By subtracting the real one from the calculated one is found the error. From it the next steps will try to find where is this error. 

In [ ]:
fs = 100 # Hz
COPX = np.zeros((49, 60*fs-30))
COPZ = np.zeros((49, 60*fs-30))

COPXreal = np.zeros((49, 60*fs-30))
COPZreal = np.zeros((49, 60*fs-30))

d1x = np.zeros((49, 60*fs-30))
d2x = np.zeros((49, 60*fs-30))

d1z = np.zeros((49, 60*fs-30))
d2z = np.zeros((49, 60*fs-30))

matrizangx = np.zeros((49, 60*fs-30))
matrizangz = np.zeros((49, 60*fs-30))


for s in range (1,50):
    a = PDSinfo[PDSinfo['Subject']==s].index.tolist()[2]
    
    fname_grf = os.path.join(path2, PDSinfo.Trial[a] + 'grf' + '.txt') 
    fname_mkr = os.path.join(path2, PDSinfo.Trial[a] + 'mkr' + '.txt')
    grf = pd.read_csv(fname_grf, delimiter='\t', header=0, engine='c')
    mkr = pd.read_csv(fname_mkr, delimiter='\t', header=0, engine='c')
    COGX=mkr['COG_X']
    COGY=mkr['COG_Y']
    COGZ=mkr['COG_Z']
    mass=PDSinfo['Mass']
    d = np.sqrt(COGX**2+COGY**2+COGZ**2)
    angulox = np.arcsin(COGX/d)
    anguloz= np.arcsin(COGZ/d)
    sinx=np.sin(angulox)
    sinz=np.sin(anguloz)
    cosx=np.cos(angulox)
    cosz=np.cos(anguloz)
    tanx=np.tan(angulox)
    tanz=np.tan(anguloz)
    matrizangx[s-1,:] = angulox.values[30:]
    matrizangz[s-1,:] = anguloz.values[30:]
    
    
    Jb=(4/3)*(mass.values[s])*((d[30:])**2)
    t=grf['Time']
    dt=t[1]-t[0]

    d1x[s-1,:]=np.gradient(matrizangx[s-1,:],dt)
    d1z[s-1,:]=np.gradient(matrizangz[s-1,:],dt)

    d2x[s-1,:]=np.gradient(d1x[s-1,:],dt)
    d2z[s-1,:]=np.gradient(d1z[s-1,:],dt)
    
    
    
    COPX[s-1,:] = d.values[30:]*sinx.values[30:] + Jb/(mass.values[s]*9.8)*(sinx.values[30:]*d1x[s-1,:]**2-cosx.values[30:]*d2x[s-1,:])
    COPZ[s-1,:] = d.values[30:]*sinz.values[30:] + Jb/(mass.values[s]*9.8)*(sinz.values[30:]*d1z[s-1,:]**2-cosz.values[30:]*d2z[s-1,:])
    
    COPXreal[s-1,:]=grf['COPNET_X'].values[30:]
    COPZreal[s-1,:]=grf['COPNET_Z'].values[30:]
    
    
errorX = COPX - COPXreal


## Graphics between real and calculated COP

In [ ]:
for s in range (1,50):
    plt.figure()
    plt.plot(COPX[s-1,:],color='blue')
    plt.plot(COPXreal[s-1,:],color='purple')
    plt.title('Comparison between COP real and predicted of Subject {}'.format(s))
    plt.show()

# COP in function of angles

Let's se how the COP correlates with the angles x and z, if the number is closer to 0 than to 1 or -1, the two variables dont correlate. From the graphics we can notice some lack of correlation between them. 

# Correlation between COPX and Angle x

In [ ]:
for s in range(1,50):
    plt.figure()
    plt.plot(COPX[s-1,:],matrizangx[s-1,:])
    plt.xlabel("Angle ")
    plt.ylabel("COPX")
    plt.title("Correlation between COPX and Angle x ")
    plt.show()

    r=np.corrcoef(matrizangx[s-1,:],COPX[s-1,:])
    print(r)
    plt.figure()
    data={'x':matrizangx[s-1,:],
          'y': COPX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

## Correlation between COPX and anglez

In [ ]:
for s in range(1,50):
    plt.figure()
    plt.plot(COPX[s-1,:],matrizangz[s-1,:])
    plt.xlabel("Angle z ")
    plt.ylabel("COPX")
    plt.title("Correlation between COPX and Angle z ")
    plt.show()

    r=np.corrcoef(matrizangx[s-1,:],COPX[s-1,:])
    print(r)
    plt.figure()
    data={'x':matrizangz[s-1,:],
          'y': COPX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

# Error in function of angles

We are going to correlate the error found with angle x and z. If the number is closer to 0 than to 1 or -1, the two variables dont correlate. From the graphics we can notice that the number of correlation approaches -1, indicating a negative correlation betweem them, for the errorz and anglez. With angle x it's possible to conclude that they dont correlate. 

## Correlation between errorx and angle x 

In [ ]:
for s in range(1,50):
    plt.figure()
    plt.plot(matrizangx[s-1,:], errorX[s-1,:])
    plt.xlabel("Angle x ")
    plt.ylabel("Error X")
    plt.title("Correlation between Error x and Angle x")
    plt.show()

    r=np.corrcoef(matrizangx[s-1,:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':matrizangx[s-1,:],
          'y': errorX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

## Correlation between errorx and anglez

In [ ]:
for s in range(1,50):
    plt.figure()
    plt.plot(matrizangz[s-1,:],errorX[s-1,:])
    plt.xlabel("Angle z ")
    plt.ylabel("Error X")
    plt.title("Correlation between Error X and Angle z ")
    plt.show()

    r=np.corrcoef(matrizangx[s-1,:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':matrizangz[s-1,:],
          'y': errorX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

# Error in function of trigonometry

We are going to correlate the error found with the trigonometry angle x and z. If the number is closer to 0 than to 1 or -1, the two variables dont correlate. From the graphics we can notice that the number of correlation approaches -1, inidicating a negative correlation betweem them, for the errorz and trigonometry of the anglez. With angle x it's possible to conclude that they dont correlate. 

## Error x and senx, cosx and tanx

In [ ]:
for s in range(1,50):
    plt.figure()
    plt.plot(sinx[30:],errorX[s-1,:],color='blue',label='Sen x')
    plt.plot(cosx[30:],errorX[s-1,:],color='red',label='Cos x')
    plt.plot(tanx[30:],errorX[s-1,:],color='purple',label='Tan x')
    plt.xlabel("Trigonometry ")
    plt.ylabel("Error x")
    plt.legend()
    plt.show()

    plt.plot(sinx[30:],errorX[s-1,:])
    plt.xlabel("Sin x")
    plt.ylabel("Error x")
    plt.title("Correlation between error x and Sin x")
    plt.show()
    r=np.corrcoef(sinx[30:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':sinx[30:],
          'y': errorX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

    plt.plot(cosx[30:],errorX[s-1,:])
    plt.xlabel("Cos x")
    plt.ylabel("Error x")
    plt.title("Correlation between error x and Cos x")
    plt.show()
    r=np.corrcoef(cosx[30:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':cosx[30:],
          'y': errorX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

    plt.plot(tanx[30:],errorX[s-1,:])
    plt.xlabel("Tan x")
    plt.ylabel("Error x")
    plt.title("Correlation between error x and Tan x")
    plt.show()
    r=np.corrcoef(tanx[30:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':tanx[30:],
          'y': errorX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

## Correlation between errorx and sen,cos and tan of z

In [ ]:
for s in range(1,50):
    plt.figure()
    plt.plot(sinz[30:],errorX[s-1,:],color='blue',label='Sen z')
    plt.plot(cosz[30:],errorX[s-1,:],color='red',label='Cos z')
    plt.plot(tanz[30:],errorX[s-1,:],color='purple',label='Tan z')
    plt.xlabel("Trigonometry ")
    plt.ylabel("Error x")
    plt.legend()
    plt.show()

    plt.plot(sinz[30:],errorX[s-1,:])
    plt.xlabel("Sin z")
    plt.ylabel("Error x")
    plt.title("Correlation between error x and Sin z")
    plt.show()
    r=np.corrcoef(sinz[30:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':sinz[30:],
          'y': errorX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

    plt.plot(cosz[30:],errorX[s-1,:])
    plt.xlabel("Cos z")
    plt.ylabel("Error x")
    plt.title("Correlation between error x and Cos z")
    plt.show()
    r=np.corrcoef(cosz[30:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':cosz[30:],
          'y': errorX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

    plt.plot(tanz[30:],errorX[s-1,:])
    plt.xlabel("Tan z")
    plt.ylabel("Error x")
    plt.title("Correlation between error x and Tan z")
    plt.show()
    r=np.corrcoef(tanz[30:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':tanz[30:],
          'y': errorX[s-1,:]
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()

# Error x in function of derivatives

We are going to correlate the error found with the first and second. If the number is closer to 0 than to 1 or -1, the two variables dont correlate. From the graphics we can notice that the number of correlation approaches 0, indicating a lack of correlation betweem them, for the errorz and anglez and angle x for the first and second derivative.

## Correlation between errorx and first and second derivative of x

In [ ]:
import seaborn as sns
for s in range(1,50):
    plt.figure()
    plt.plot(d1x[s-1,:],errorX[s-1,:],color='blue',label='First derivative')
    plt.plot(d2x[s-1,:],errorX[s-1,:],color='red',label='Second derivative')
    plt.xlabel("Derivative")
    plt.ylabel("Error x")
    plt.legend()
    plt.show()

    plt.plot(d1x,errorX,'bo')
    plt.xlabel("First derivative x")
    plt.ylabel("Errorx")
    plt.title("Correlation between error x and First derivative x ")
    plt.show()
    r=np.corrcoef(d1x[s-1,:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':d1x[s-1,:],
          'y': errorX[s-1,:],
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()
    plt.plot(d2x[s-1,:],errorX[s-1,:])
    plt.xlabel("Second derivative x")
    plt.ylabel("Error x")
    plt.title("Correlation between error x and Second deivative  x ")
    plt.show()
    r=np.corrcoef(d2x[s-1,:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':d2x[s-1,:],
          'y': errorX[s-1,:],
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()
    

## Correlation between errorx and first and second derivative of z

In [ ]:
for s in range(1,50):
    plt.figure()
    plt.plot(d1z[s-1,:],errorX[s-1,:],color='blue',label='First derivative')
    plt.plot(d2z[s-1,:],errorX[s-1,:],color='red',label='Second derivative')
    plt.xlabel("Derivative")
    plt.ylabel("Error x")
    plt.legend()
    plt.show()

    plt.plot(d1z[s-1,:],errorX[s-1,:],'bo')
    plt.xlabel("First derivative z")
    plt.ylabel("Errorx")
    plt.title("Correlation between error x and First derivative z")
    plt.show()
    r=np.corrcoef(d1z[s-1,:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':d1z[s-1,:],
          'y': errorX[s-1,:],
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()
    plt.plot(d2z[s-1,:],errorX[s-1,:])
    plt.xlabel("Second derivative z")
    plt.ylabel("Error x")
    plt.title("Correlation between error x and Second deivative z ")
    plt.show()
    r=np.corrcoef(d2z[s-1,:],errorX[s-1,:])
    print(r)
    plt.figure()
    data={'x':d2z[s-1,:],
          'y': errorX[s-1,:],
         }
    df=pd.DataFrame(data)
    correlation=df.corr()
    plot=sns.heatmap(correlation,annot= True, fmt='.1f',linewidths=.6)
    plt.show()


In [ ]:
for s in range(1,50):
    plt.figure()
    hist=plt.hist(errorZ[s-1,:])
    plt.show()